<h1> Máquinas de aprendizaje: Tarea 1


Felipe Araya Barrera - 201173515-3  
Sebastián Vergara Miranda - 201173515-3  
9 de septiembre de 2016

<h1> Importaciones necesarias (secciones 1, 2 y 3)

In [13]:
#!usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as lm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import cross_validation
import matplotlib.pyplot as plt
import scipy.stats as stats

<h1> 1. Regresión Lineal Ordinaria (LSS)

**1.1 Construcción de dataframe**

Se construye un dataframe con los datos provistos:

In [6]:
url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)

Luego, se realizan los siguientes ajustes sobre este:

1. Se elimina columna sin nombre que especifica la posición de cada registro en el dataframe (línea 5). <br> <br>
  
2. Se almacena por separado la variable *train*, la cual indica si el registro pertenece o no
al conjunto de entrenamiento (línea 6). <br> <br>

3. Se crea un arreglo que realiza un cambio de notación para los valores que puede tomar
la variable train, de tal manera que si posee el valor 'T', se cambia por **True**, mientras que si el valor es 'F' se cambia por **False** (línea 7). <br> <br>

4. Se crea un arreglo en el que se indica si cada registro pertenece o no al conjunto de
prueba (línea 8). <br> <br>

5. Se elimina la variable train del dataframe creado (línea 9).

In [4]:
df = df.drop('Unnamed: 0', axis=1)
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
istest = np.logical_not(istrain)
df = df.drop('train',axis=1)

**1.2 Descripción de dataset**

El dataset se compone de 97 registros (pacientes), cada uno de los cuáles está descrito por
9 variables, las que se detallan a continuación:

1. *lcavol* : Logaritmo del volumen de cáncer presente. <br> <br>
    
2. *lweight*: Logaritmo del peso de la próstata. <br> <br> 

3. *age*: Edad. <br> <br>  

4. *lbph*: Logaritmo de la cantidad de hiperplasia benigna de próstata. <br> <br> 

5. *svi*: Indica si existe invasión de la vesícula seminal o no. <br> <br> 

6. *lcp*: Logaritmo de la penetración capsular. <br> <br>

7. *gleason*: Medida del grado de agresividad del cáncer, en base a la escala de Gleason. <br> <br>  

8. *pgg45* : Porcentaje que representa la presencia de los patrones de Gleason 4 y 5. <br> <br>  

9. *lpsa*: Logaritmo del nivel de antígeno prostático específico (PSA). <br> <br>
Además, no existen valores nulos para ningún registro. Más información acerca del dataset (principalmente estadística) puede encontrarse al ejecutar los siguientes comandos: 

In [10]:
df.info()

<bound method DataFrame.info of     Unnamed: 0    lcavol   lweight  age      lbph  svi       lcp  gleason  \
0            1 -0.579818  2.769459   50 -1.386294    0 -1.386294        6   
1            2 -0.994252  3.319626   58 -1.386294    0 -1.386294        6   
2            3 -0.510826  2.691243   74 -1.386294    0 -1.386294        7   
3            4 -1.203973  3.282789   58 -1.386294    0 -1.386294        6   
4            5  0.751416  3.432373   62 -1.386294    0 -1.386294        6   
5            6 -1.049822  3.228826   50 -1.386294    0 -1.386294        6   
6            7  0.737164  3.473518   64  0.615186    0 -1.386294        6   
7            8  0.693147  3.539509   58  1.536867    0 -1.386294        6   
8            9 -0.776529  3.539509   47 -1.386294    0 -1.386294        6   
9           10  0.223144  3.244544   63 -1.386294    0 -1.386294        6   
10          11  0.254642  3.604138   65 -1.386294    0 -1.386294        6   
11          12 -1.347074  3.598681   63  1.2

In [11]:
df.describe()

,Unnamed: 0,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,49.000000,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,28.145456,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,1.000000,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,25.000000,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,49.000000,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,73.000000,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,97.000000,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


**1.3 Normalización de datos**

Antes de llevar la construcción de un modelo predictivo, es necesario normalizar los datos,
pues se está trabajando con variables medidas en unidades y escalas diferentes. Al normalizar,
es posible realizar comparaciones razonables entre ellas. Así, cada variable numérica se escala
en el rango [0,1]. Para esta instancia en particular, los valores de *lpsa* se mantienen respecto al dataset original. También, se observa que la media de cada variable tiende a 0 y la varianza es muy cercana a 1. Para la normalización, se lleva a cabo el siguiente procedimiento:

In [14]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['lpsa'] = df['lpsa']

ValueError: could not convert string to float: F

**1.4 Construcción de modelo de regresión lineal ordinaria**

Para implementar el modelo, se ejecutan los siguientes pasos: <br> 
1. Primero, se separa el conjunto de datos en dos partes: La primera contiene solo las
variables predictivas ($C_1$), mientras que la segunda, sólo la variable a predecir ($C_2$).<br> <br> 
2. En $C_1$ , se agrega una nueva variable, que representa el intercepto que será utilizado por
cada registro para construir el modelo de regresión. <br> <br>
3. Tanto $C_1$ como $C_2$ se subdividen en un conjunto que contiene registros que pertenecen
al set de entrenamiento ($C_{11}$ y $C_{21}$) y en otro que agrupa los que pertenecen al set de prueba ($C_{21}$ y $C_{22}$ ). <br> <br>
4. Finalmente, para implementar la regresión lineal, se entrega a la función correspondiente
los conjuntos $C_{11}$ y $C_{21}$ como argumentos.

In [16]:
X = df_scaled.ix[: , :-1]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['lpsa']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)

NameError: name 'df_scaled' is not defined

**1.5 Pesos y Z-score**

Se obtienen los pesos, errores estándar y Z-score para cada variable del
modelo elaborado.

In [17]:
#Pesos
weights = linreg.coef_
#Error estandar
SEM = np.asarray(Xtrain.std()) / np.sqrt(len(Xtrain))
#A partir de lo anterior, se calculan los Z-score de cada variable
Z_score = weights / SEM

NameError: name 'linreg' is not defined

La tabla 1 resume los valores obtenidos.

 <table style="width:100%">
 <caption>Tabla 1: Peso, SEM y Z-score de cada variable del modelo LSS.</caption>
  <tr>
    <th>**Variable**</th>
    <th>**Peso**</th>
    <th>**Error Estándar (SEM)**</th>
    <th>**Z-score**</th>
  </tr>
  <tr>
    <td>*lcavol*</td>
    <td>0.68</td>
    <td>0.13</td>
    <td>5.22</td>
  </tr>
  <tr>
    <td>*lweight*</td>
    <td>0.26</td>
    <td>0.14</td>
    <td>1.92</td>
  </tr>
  <tr>
    <td>*lcavol*</td>
    <td>-0.14</td>
    <td>0.12</td>
    <td>-1.14</td>
  </tr>
  <tr>
    <td>*lbph*</td>
    <td>0.21</td>
    <td>0.12</td>
    <td>1.69</td>
  </tr>
  <tr>
    <td>*svi*</td>
    <td>0.30</td>
    <td>0.12</td>
    <td>2.44</td>
  </tr>
  <tr>
    <td>*lcp*</td>
    <td>-0.29</td>
    <td>0.12</td>
    <td>-2.33</td>
  </tr>
  <tr>
    <td>*gleason*</td>
    <td>-0.02</td>
    <td>0.12</td>
    <td>-0.18</td>
  </tr>
  <tr>
    <td>*pgg45*</td>
    <td>0.27</td>
    <td>0.13</td>
    <td>2.08</td>
  </tr>
</table> 

De acuerdo a estos resultados, las variables que presentan una mayor correlación con la
variable a predecir son *lcavol*, *svi* y *lcp*. Si se utiliza un nivel de significancia del 5 %, entonces existe evidencia suficiente para afirmar que la variable **NO** presenta una fuerte relación con la respuesta si Z-score ∈ [-1.668, 1.668]. Ante esto, existe evidencia suficiente para afirmar que las variables *age* y *gleason* no están relacionadas con la variable a predecir.

**1.6 Estimación de error de predicción**

Primeramente, se calcula el error real del modelo (error cuadrático medio), obteniéndose
MSE = 0.52.

In [18]:
yhat_test = linreg.predict(Xtest)
mse_test = np.mean(np.power(yhat_test - ytest, 2))
print mse_test

NameError: name 'linreg' is not defined

Luego, se evalúan los resultados obtenidos por medio del método cross validation.
Se estudian dos casos:

**Caso 1**: K = 5

In [19]:
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
k_fold1 = cross_validation.KFold(len(Xm), 5)
mse_cv1 = 0
for k, (train, val) in enumerate(k_fold1):
  linreg = lm.LinearRegression(fit_intercept = False)
  linreg.fit(Xm[train], ym[train])
  yhat_val = linreg.predict(Xm[val])
  mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
  mse_cv1 += mse_fold
mse_cv1 = mse_cv1 / 5
print mse_cv1

NameError: name 'Xtrain' is not defined

Así, para K = 5, se obtiene un error $e_1$ = 0.96.

**Caso 2**: K = 10

In [21]:
k_fold2 = cross_validation.KFold(len(Xm), 10)
mse_cv2 = 0
for k, (train, val) in enumerate(k_fold2):
  linreg = lm.LinearRegression(fit_intercept = False)
  linreg.fit(Xm[train], ym[train])
  yhat_val = linreg.predict(Xm[val])
  mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
  mse_cv2 += mse_fold
mse_cv2 = mse_cv2 / 10
print mse_cv2

NameError: name 'Xm' is not defined

Es decir, con K = 10, se obtiene un error $e_2$ = 0.76. <br>

Se observa que para los casos 1 y 2, el error es considerablemente mayor respecto a MSE.
Esto quiere decir que existe un alto nivel de dependencia del modelo respecto a los datos
usados para construirlo. En otras palabras, el modelo está sobre ajustado (*overfitting*), por
lo que entregará buenas predicciones para los casos pertenecientes al dataset original, pero
no será el más apropiado si se desea predecir el valor de la variable de interés para casos de
pacientes que no se encuentran dentro del conjunto de datos de entrenamiento.

**1.7 Error de predicción por dato**

Se estudia la validez de suponer que los errores de predicción sobre los datos de entrenamiento siguen una distribución normal. Con la ejecución del siguiente código, se obtiene un gráfico que muestra el comportamiento de los errores:

In [23]:
#Se estima error de prediccion por cada dato de entrenamiento
yhat_train = linreg.predict(Xtrain)
ytrain_array = np.asarray(ytrain)
error = yhat_train - ytrain_array
#Se genera grafico de errores
stats.probplot(error, dist='norm', plot=plt)
plt.title('Siguen los errores de prediccion sobre el conjunto de entrenamiento una distribucion normal?')
plt.ylabel('Error dato de entrenamiento')
plt.show()

NameError: name 'linreg' is not defined

Se observa que el conjunto de errores puede representarse mediante una recta, idea que
se reafirma al notar que el coeficiente de correlación es $R^2$ = 0,9913. Como consecuencia de
lo anterior, y de acuerdo con el marco teórico, es correcto señalar que los errores siguen una
distribución normal.

<h1>2. Selección de atributos

**Selección vía FSS**

Se implementa *Forward Step-wise Selection* (FSS). Para agregar una variable al modelo, se utiliza como criterio el mínimo MSE. A continuación, se construye un gráfico con el error de entrenamiento y de pruebas de acuerdo en función de la cantidad de variables. Es pertinente señalar que FSS parte incluyendo el intercepto y luego empieza a agregar variables.